In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [2]:
rating = pd.read_csv('/Users/blakemyers/Desktop/data/ratings.csv', error_bad_lines=False, encoding='latin-1')

In [3]:
movie = pd.read_csv("/Users/blakemyers/Desktop/data/movies.csv", error_bad_lines=False, encoding='latin-1')

In [4]:
movie_rating = pd.merge(rating, movie, on = 'movieId')

In [5]:
cols = ['timestamp']

In [6]:
movie_rating.drop(cols, axis=1, inplace=True)

In [7]:
numrate_movie = movie_rating.groupby("title")["rating"].count().reset_index()

In [8]:
numrate_movie.rename({"rating": "ratecount_movie"}, axis=1, inplace=True)

In [9]:
numrate_movie = numrate_movie.query("ratecount_movie >= 20")

In [10]:
ratings20plus = pd.merge(numrate_movie, movie_rating, on = 'title', how = 'inner')

In [11]:
numrate_user = ratings20plus.groupby("userId")["rating"].count().reset_index()

In [12]:
numrate_user.rename({"rating": "ratecount_user"}, axis=1, inplace=True)

In [13]:
numrate_user = numrate_user.query("ratecount_user >= 20")

In [14]:
ur20plus = pd.merge(ratings20plus, numrate_user, on = "userId", how = "inner")

In [15]:
ur20plus = ur20plus.append(pd.DataFrame([["A.I. Artificial Intelligence (2001)",1,9999999,4370,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)

In [16]:
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled

In [17]:
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)

In [18]:
user_movie_matrix.head()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2001: A Space Odyssey (1968),2012 (2009),...,Young Frankenstein (1974),Young Guns (1988),Zack and Miri Make a Porno (2008),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),Â¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.777778,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.777778
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.555556,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [19]:
X_train, X_test = train_test_split(user_movie_matrix, train_size=0.8)

In [20]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
num_input = ur20plus['title'].nunique()
n_nodes_inpl = num_input  
n_nodes_hl1  = 256  
n_nodes_outl = num_input  
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl]))}

In [22]:
input_layer = tf.placeholder('float', [None, num_input])
# add a constant node to the first layer
# it needs to have the same shape as the input layer for me to be
# able to concatinate it later
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,hidden_1_layer_vals['weights']))
# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, num_input])
# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define our optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# defining batch size, number of epochs and learning rate
batch_size = 100  # how many images to use together for training
hm_epochs =200    # how many times to go through the entire dataset
tot_images = X_train.shape[0] # total number of images

In [24]:
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    
    for i in range(int(tot_images/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer,\
               feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer,\
                   feed_dict={input_layer:X_test})
        
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

MSE train 89.96492036064706 MSE test 91.40298170226646
Epoch 0 / 200 loss: 421.17105865478516
MSE train 74.49256635097204 MSE test 75.73179818294422
Epoch 1 / 200 loss: 335.15612030029297
MSE train 64.42942806256198 MSE test 65.57300277497373
Epoch 2 / 200 loss: 282.2219772338867
MSE train 57.48358917444341 MSE test 58.590559079518535
Epoch 3 / 200 loss: 247.02777099609375
MSE train 52.46268154015393 MSE test 53.547939670047676
Epoch 4 / 200 loss: 222.34178924560547
MSE train 48.69304905843875 MSE test 49.78673178749168
Epoch 5 / 200 loss: 204.25551223754883
MSE train 45.77716343558368 MSE test 46.89450244211063
Epoch 6 / 200 loss: 190.52746200561523
MSE train 43.46722333565716 MSE test 44.620625653979374
Epoch 7 / 200 loss: 179.8059310913086
MSE train 41.59448060705847 MSE test 42.791357474875156
Epoch 8 / 200 loss: 171.23655319213867
MSE train 40.04200060868408 MSE test 41.290264572092916
Epoch 9 / 200 loss: 164.22450637817383
MSE train 38.73138475442937 MSE test 40.035988303675
Epoc

MSE train 14.027994679261921 MSE test 16.058291366632005
Epoch 87 / 200 loss: 55.60274791717529
MSE train 13.881930059329957 MSE test 15.910095246608494
Epoch 88 / 200 loss: 55.01607608795166
MSE train 13.737130376491967 MSE test 15.763841196733834
Epoch 89 / 200 loss: 54.43508529663086
MSE train 13.593617294911649 MSE test 15.619552694383177
Epoch 90 / 200 loss: 53.85881519317627
MSE train 13.451745836040473 MSE test 15.47725218305117
Epoch 91 / 200 loss: 53.28773593902588
MSE train 13.311811439498719 MSE test 15.336924265526001
Epoch 92 / 200 loss: 52.72353553771973
MSE train 13.173834154344476 MSE test 15.198518346634645
Epoch 93 / 200 loss: 52.16720485687256
MSE train 13.037729374652594 MSE test 15.062006391824248
Epoch 94 / 200 loss: 51.61858558654785
MSE train 12.903370428510087 MSE test 14.92736604297503
Epoch 95 / 200 loss: 51.07716941833496
MSE train 12.770560570234192 MSE test 14.794560147596862
Epoch 96 / 200 loss: 50.5423698425293
MSE train 12.639181132383946 MSE test 14.66

MSE train 6.764302983353759 MSE test 8.831061015365627
Epoch 173 / 200 loss: 26.13259506225586
MSE train 6.725418187697144 MSE test 8.79299569507986
Epoch 174 / 200 loss: 25.975111961364746
MSE train 6.686934640102498 MSE test 8.755356041509739
Epoch 175 / 200 loss: 25.819321632385254
MSE train 6.6488399095492055 MSE test 8.718119340442874
Epoch 176 / 200 loss: 25.665165424346924
MSE train 6.611122688543493 MSE test 8.681263515619518
Epoch 177 / 200 loss: 25.512583255767822
MSE train 6.573772734583093 MSE test 8.644769229298992
Epoch 178 / 200 loss: 25.36152696609497
MSE train 6.536783415807143 MSE test 8.608621468475013
Epoch 179 / 200 loss: 25.211952209472656
MSE train 6.500157270003603 MSE test 8.572812138417333
Epoch 180 / 200 loss: 25.063830375671387
MSE train 6.463911244604017 MSE test 8.537339915483619
Epoch 181 / 200 loss: 24.91718339920044
MSE train 6.428071790897124 MSE test 8.502207491998767
Epoch 182 / 200 loss: 24.77208709716797
MSE train 6.392656575626783 MSE test 8.46741

In [25]:
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)

In [26]:
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999999]

,userId,title,rating
735066,9999999,Ronin (1998),8.881796
734748,9999999,"Knight's Tale, A (2001)",8.210431
734373,9999999,Cowboy Bebop: The Movie (Cowboy Bebop: Tengoku...,7.323219
734857,9999999,Miller's Crossing (1990),7.304662
734651,9999999,Home Alone 2: Lost in New York (1992),7.096229
734283,9999999,Brave (2012),7.069924
735327,9999999,"Village, The (2004)",6.744802
734783,9999999,Little Big Man (1970),6.672563
735329,9999999,WALLÂ·E (2008),6.598843
734523,9999999,"Fish Called Wanda, A (1988)",6.533463


In [27]:
ur20plus = ur20plus.append(pd.DataFrame([["Hulk (2003)",1,9999991,4370,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)

In [28]:
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)

In [29]:
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999991]

,userId,title,rating
734670,9999991,"Hunt for Red October, The (1990)",11.114067
734510,9999991,"Fifth Element, The (1997)",10.140924
734767,9999991,Legally Blonde (2001),9.512592
735244,9999991,The Devil's Advocate (1997),9.327778
735062,9999991,Role Models (2008),8.602976
734525,9999991,"Fistful of Dollars, A (Per un pugno di dollari...",8.517904
734149,9999991,Aliens (1986),8.471788
735120,9999991,Sherlock Holmes: A Game of Shadows (2011),8.281423
735355,9999991,West Side Story (1961),8.238237
734123,9999991,"6th Day, The (2000)",8.189003


In [30]:
user_movie_matrix.tail()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2001: A Space Odyssey (1968),2012 (2009),...,Young Frankenstein (1974),Young Guns (1988),Zack and Miri Make a Porno (2008),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),Â¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
608,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.111111,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.111111,0.000000,0.177778,0.133333,0.0
609,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
610,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.155556,0.177778,0.111111,...,0.0,0.0,0.133333,0.2,0.133333,0.155556,0.155556,0.000000,0.066667,0.0
9999991,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9999999,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [31]:
ur20plus = ur20plus.append(pd.DataFrame([["Wild Wild West (1999)",1,9999995,900000000,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)

In [32]:
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999995]

,userId,title,rating
735558,9999995,"Blind Side, The (2009)",9.221283
735802,9999995,Fear and Loathing in Las Vegas (1998),9.130622
736154,9999995,Miller's Crossing (1990),8.398761
736253,9999995,"Patriot, The (2000)",8.047646
735798,9999995,Fast Times at Ridgemont High (1982),8.016894
735809,9999995,"Fighter, The (2010)",7.633740
735512,9999995,Batman Begins (2005),7.606032
736056,9999995,"Last King of Scotland, The (2006)",7.536535
735830,9999995,Forget Paris (1995),7.505000
735645,9999995,Clear and Present Danger (1994),7.322256


In [33]:
ur20plus = ur20plus.append(pd.DataFrame([["Jerk, The (1979)",1,9999996,900000000,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)

In [34]:
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999996]

,userId,title,rating
737411,9999996,Mallrats (1995),10.214937
737117,9999996,"Fish Called Wanda, A (1988)",8.504625
737289,9999996,Inside Out (2015),8.438497
737119,9999996,"Fistful of Dollars, A (Per un pugno di dollari...",8.079088
737094,9999996,Fargo (1996),7.714064
737852,9999996,"Thing, The (1982)",7.661129
737482,9999996,Mulan (1998),7.631747
736967,9999996,Cowboy Bebop: The Movie (Cowboy Bebop: Tengoku...,7.422754
737153,9999996,Gangs of New York (2002),7.361778
737690,9999996,Scream (1996),7.350254
